# MedCorpora: обработка данных

Источником данных для корпуса является раздел «Новости» сайта [«Доктор Питер»](https://doctorpiter.ru).

Тексты были собраны Анной Луценко в рамках лабораторной работы на курсе «Математика для лингвистов», который был прочитан в 4 модуле 2023-2024 учебного года.

Код, использовавшийся для сбора данных представлен по [ссылке](https://github.com/usmor/laboratory-work-1).

В данном файле представлена предобработка текстов для дальнейшего использования в проекте по созданию корпуса.

В первую очередь необходимо было разбить каждый новостной текст на предложения и записать его вместе со всей мета-информацией в отдельный csv файл.

Затем, в каждом предложении мы выделили токены (удалив при этом пунктуацию), лемматизировали их, определили часть речи и привели к нижнему регистру. В нашей таблице каждому предложению соответствует разбор, состоящий из шаблона вида «токен+лемма+тег».

Разбиение на предложения и морфологическая обработка токенов были сделаны с использованием stanza. Обработка данных заняла 3 часа 39 минут.


# Предобработка текста и разметка корпуса

Установка и импорт необходимых модулей.

In [ ]:
%pip install stanza

In [ ]:
import stanza
import pandas as pd
from tqdm import tqdm

Словарь для хранения предложений, разбора предложений и мета-информации о тексте.

In [ ]:
corpora = {
    "Текст": [],
    "Разбор": [],
    "Автор": [],
    "Дата": [],
    "Источник": [],
    "Ссылка": []
}

Функция для разбора предложения.

In [ ]:
def morphological_analisys(sentence) -> str:
    # список для разбора предложения по токенам
    result = []
    for word in sentence.words:
        # если слово не пунктуация и не спец.символ
        if word.upos.lower() != "punct" and word.upos.lower() != "sym":
            # добавляем слово, лемму и часть речи в нижнем регистре
            result.append(f"{word.text.lower()}+{word.lemma.lower()}+{word.upos.lower()}")

    return " ".join(result)


Обработка данных.

In [ ]:
with open("Doctor_Piter.txt", "r") as texts:
    # разбиваем файл на отдельные новости
    texts = texts.read().split("=====\n")

    # строим пайплайн
    nlp = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma')

    for element in tqdm(texts):
        # собираем метаинформацию по каждой новости
        url, source, date, author, name = element.split("\n")[:5]

        url = url.replace(" ", "")

        text = ' '.join(element.split("\n")[-4:])

        # делаем аннотацию текста
        doc = nlp(text)

        # разбираем каждый текст по предложениям
        for sentence in doc.sentences:
            # записываем метаинформацию в словарь
            corpora["Текст"].append(sentence.text)
            corpora["Автор"].append(author)
            corpora["Дата"].append(date)
            corpora["Источник"].append(source)
            corpora["Ссылка"].append(url)

            # делаем разбор предложения с помощью stanza
            corpora["Разбор"].append(morphological_analisys(sentence))


2024-10-22 12:52:02 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-22 12:52:03 INFO: Downloaded file to /Users/veronikatsareva/stanza_resources/resources.json
2024-10-22 13:02:26 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

2024-10-22 13:02:26 INFO: Using device: cpu
2024-10-22 13:02:26 INFO: Loading: tokenize
2024-10-22 13:02:28 INFO: Loading: pos
2024-10-22 13:02:29 INFO: Loading: lemma
2024-10-22 13:02:29 INFO: Done loading processors!
100%|██████████| 6869/6869 [3:39:36<00:00,  1.92s/it]  


Запись словаря в датафрейм.

In [ ]:
df = pd.DataFrame.from_dict(corpora)

Запись датафрейма в csv.

In [ ]:
df.to_csv('MedNewsCorpora.csv')

После того, как все тексты были разбиты на предложения, мы обнаружили, что при перечислении некоторых организаций в скобках, stanza выделяла «).» в отдельное предложение. Разбор предложения в таком случае имел тип nan.

Мы решили присоединить все такие «предложения» обратно к предыдущему, и удалить такие строки из датасета.

Чтение датафрейма.

In [ ]:
df = pd.read_csv('MedNewsCorpora.csv')

Удаление ненужного столбца.

In [ ]:
df.drop(columns="Unnamed: 0", inplace=True)

Очистка данных.

In [ ]:
for i, row in enumerate(df.itertuples()):
    # если разбор nan, то
    if not isinstance(row.Разбор, str):
        # присоединяем скобки к предыдущему предложению
        df.at[row.Index - 1, "Текст"] = f'{df.loc[row.Index - 1]["Текст"]}{df.loc[row.Index]["Текст"]}'
        # удаляем вхождение предложения из датасета
        df.drop(index=row.Index, inplace=True)

# перезаписываем индексы
df.reset_index(drop=True, inplace=True)

В нашем корпусе оказалось практически два миллиона словоформ, поэтому для более быстрой работы приложения мы решили сделать сокращенную версию и оставить только первые 10 тысяч предложений.

Создание short-версии корпуса.

In [ ]:
short_df = df[:10000]

Запись short-версии корпуса в csv.

In [ ]:
short_df.to_csv('MedNewsCorpora-short.csv')

В ходе работы выяснилось, что для мета-информации нам необходимы и названия статей, из которых были взяты статьи, но которые мы забыли изначально добавить в датафрейм.

Мы решили составить словарь соответствий, где ключ –– ссылка, а значение –– название статьи.

Затем мы прошлись по датафрейму и составили с помощью словаря список названий для каждого предложения.

После этого мы добавили новый столбец в датафрейм и перезаписали csv файл.

Составление словаря из ссылок и названий статей.

In [ ]:
with open("Doctor_Piter.txt", "r") as texts:
    texts = texts.read().split("=====\n")

    url_dict = {}

    for element in tqdm(texts):
        url, name = element.split("\n")[0], element.split("\n")[4]
        url = url.replace(" ", "")
        url_dict[url] = name

100%|██████████| 6869/6869 [00:00<00:00, 146976.74it/s]


Чтение csv-файла.

In [ ]:
short_df = pd.read_csv('MedNewsCorpora-short.csv')

In [ ]:
short_df.drop(columns="Unnamed: 0", inplace=True)

Составление списка названий статей для каждого предложения.

In [ ]:
names = [url_dict[url] for url in df["Ссылка"]]

Добавление нового столбца в датафрейм.

In [ ]:
short_df_names = pd.DataFrame.from_dict({"Название статьи" : names})

In [ ]:
short_df = pd.concat([short_df, short_df_names], axis=1)

Перезапись датафрейма.

In [ ]:
short_df.to_csv('MedNewsCorpora-short.csv')